# Generate pressure coefficients

The following steps are performed to obtain all the artifacts for pressure coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data
   * For reference static pressure
   * For body pressure data
   * Slice from timestep range specified
4. Transform pressure data into pressure coefficient
5. Calculate pressure coefficient statistics
6. Merge coefficient statistics with mesh description into VTK
  
  

Read post-processing Cp config file

In [1]:
from cfdmod.use_cases.pressure.cp_config import CpConfig
import pathlib

post_proc_cfg_path = pathlib.Path("./fixtures/tests/pressure/cp_params.yaml") 
post_proc_cfg = CpConfig.from_file(post_proc_cfg_path)

post_proc_cfg

CpConfig(timestep_range=(10000.0, 20000.0), reference_pressure='average', U_H=0.05, statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis'])

Read simulation config file

In [2]:
from nassu.cfg.model import ConfigScheme

sim_cfg_path = pathlib.Path("A:/Aerosim/Repositories/nassu/tests/validation/results/test_simulation/setup_nassu_case_v1.5.yaml")

sim_cfg = ConfigScheme.from_file(sim_cfg_path).load_sim_cfgs()[0]

sim_cfg

NassuOSError: Filename A:/Aerosim/Repositories/nassu/tests/validation/results/test_simulation/setup_nassu_case_v1.5.yaml not found. Unable to load configurations

Read LNAS Mesh

In [ ]:
from nassu.lnas import LagrangianFormat

body_cfg = sim_cfg.domain.bodies["galpao"]

mesh = LagrangianFormat.from_folder(body_cfg.lagrangian_path)

mesh.name, mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

('galpao', (1549, 3), (2915, 3))

Get data path from simulation configs

In [ ]:
from nassu.utils import Environment
import pandas as pd

root_path = pathlib.Path("../nassu")

Environment.set_path(pathlib.Path("../nassu/.env"))

sim_output = sim_cfg.output

press_data_path = root_path / sim_output.series["default"].points["static_pressure"].data_filename
body_data_path = root_path / sim_output.series["default"].bodies["galpao"].data_filename

press_data_path, body_data_path

[2023-10-06 14:34:50.27] [INFO] Updated '.env' path to WindowsPath('../nassu/.env')


(WindowsPath('../nassu/tests/validation/results/test_simulation/test_simulation/000/probes/hist_series/default/points.static_pressure.data.h5'),
 WindowsPath('../nassu/tests/validation/results/test_simulation/test_simulation/000/probes/hist_series/default/bodies.galpao.data.h5'))

Read hist series and slice from the time range

In [ ]:
from cfdmod.use_cases.pressure.cp_data import filter_pressure_data

press_data: pd.DataFrame = pd.read_hdf(press_data_path) # type: ignore
body_data: pd.DataFrame = pd.read_hdf(body_data_path) # type: ignore

press_data, body_data = filter_pressure_data(
    press_data, body_data, post_proc_cfg.timestep_range
)

press_data, body_data

(      point_idx       rho  time_step
 2915          0  1.002515    10000.0
 2915          0  1.002532    10001.0
 2915          0  1.002549    10002.0
 2915          0  1.002566    10003.0
 2915          0  1.002583    10004.0
 ...         ...       ...        ...
 2915          0  1.002908    19996.0
 2915          0  1.002887    19997.0
 2915          0  1.002865    19998.0
 2915          0  1.002845    19999.0
 2915          0  1.002825    20000.0
 
 [10001 rows x 3 columns],
       point_idx       rho  time_step
 0             0  1.001819    10000.0
 1             1  1.001838    10000.0
 2             2  1.001808    10000.0
 3             3  1.001851    10000.0
 4             4  1.001835    10000.0
 ...         ...       ...        ...
 2910       2910  1.004277    20000.0
 2911       2911  1.004619    20000.0
 2912       2912  1.004608    20000.0
 2913       2913  1.004116    20000.0
 2914       2914  1.004155    20000.0
 
 [29152915 rows x 3 columns])

Transform pressure data into coefficient

In [ ]:
from cfdmod.use_cases.pressure.cp_data import transform_to_cp

cp_data = transform_to_cp(
    press_data,
    body_data,
    reference_vel=post_proc_cfg.U_H,
    ref_press_mode=post_proc_cfg.reference_pressure,
)

cp_data

,time_step,point_idx,cp
0,10000.0,0,0.268273
1,10000.0,1,0.273546
2,10000.0,2,0.265319
3,10000.0,3,0.276913
4,10000.0,4,0.272593
...,...,...,...
29152910,20000.0,2910,0.923266
29152911,20000.0,2911,1.014458
29152912,20000.0,2912,1.011409
29152913,20000.0,2913,0.880417


Calculate pressure coefficient statistics

In [ ]:
from cfdmod.use_cases.pressure.cp_data import calculate_statistics

cp_stats = calculate_statistics(cp_data, statistics_to_apply=post_proc_cfg.statistics)

cp_stats

,point_idx,cp_avg,cp_min,cp_max,cp_rms,cp_skewness,cp_kurtosis
0,0,-0.021076,-2.392591,2.145996,0.975655,0.007537,-0.644743
1,1,-0.019863,-2.389510,2.147012,0.974600,0.005196,-0.641696
2,2,-0.021279,-2.394671,2.139135,0.975093,0.006027,-0.645222
3,3,-0.019417,-2.387731,2.152889,0.975021,0.005858,-0.640871
4,4,-0.020308,-2.389669,2.148188,0.974958,0.006443,-0.642380
...,...,...,...,...,...,...,...
2910,2910,0.065866,-2.437123,2.166579,0.996299,-0.073992,-0.699826
2911,2911,0.025314,-2.499840,2.120172,0.984458,-0.098281,-0.592226
2912,2912,0.032308,-2.501142,2.110929,0.985440,-0.100940,-0.606598
2913,2913,0.078103,-2.352251,2.193546,1.003240,-0.043307,-0.751864


Combine statistics with mesh and create a VTK object

In [ ]:
from cfdmod.api.vtk.write_vtk import write_polydata, create_polydata_for_cell_data

output_path = pathlib.Path("./output/pressure")
vtk_path = output_path / "cp_stats.vtp"

polydata = create_polydata_for_cell_data(data=cp_stats, mesh=mesh.geometry)

# OUTPUT 3: VTK cp_stats
write_polydata(vtk_path, polydata)